В текущем примере хотелось бы сделать небольшое введение в возможности SageMath по работе с формулами.<br>
Задание заключается в следующем:<br>
Вывести формулу для движения спутника (далее ИСЗ - искусственный спутник Земли) в невращающейся системе координат с центром в центре Земли (далее - геостац. СК или geostat).<br>
Орбита эллиптическая, точка отсчета для орбитальной СК находится в фокусе. В этом же фокусе находится и центр Земли.
Плоскость орбиты относительно плоскости эклиптики повернута на три угла: psi, theta, phi.<br>
Параметры орбиты (эллипса):<br>
a - большая полуось<br>
e - эксцентриситет.<br>
Для e главное не забывать, что есть математическая константа e и их можно перепутать (например, просто забыв объявить переменную e - будет везде использоваться числовое значение мат. константы e и результат не будет достигнут).

Первый шаг получения формулы - получить координаты ИСЗ в орбитальной системе координат.<br>
Движение идет постоянно в плоскости орбиты, потому z == 0. <br>
x и y задаются стандартным параметрическим уравнением эллипса:<br>
$x = a * cos(t)$<br>
$y = b * sin(t)$
<br>
b - это малая полуось эллипса. Через эксцентриситет и большую полуось выражается формулой:<br>
$a*\sqrt{1 - e^2}$<br>
Сказано, что точка отсчета смещена в фокус. Это даёт смещение по оси X на $a*e$

Итого, получим:

In [1]:
def satellite_pos_orbital_crd(_orbit_a, _orbit_e, _Xi):
    x = _orbit_a*(cos(_Xi) - _orbit_e)
    y = _orbit_a*sqrt(1 - _orbit_e**2)*sin(_Xi)
    z = 0
    
    return [x, y, z]

Теперь нужно из орбитальной СК перевести в геостац. СК.<br>
Для этого воспользуемся матрицами поворота:

In [2]:
def rot_mat(axis, angle):
    if axis == 'x':
        return Matrix([[1, 0, 0], [0, cos(angle), sin(angle)], [0, -sin(angle), cos(angle)]])
    if axis == 'y':
        return Matrix([[cos(angle), 0, sin(angle)], [0, 1, 0],  [-sin(angle), 0, cos(angle)]])
    if axis == 'z':
        return Matrix([[cos(angle), sin(angle), 0], [-sin(angle), cos(angle), 0], [0, 0, 1]])
    return Matrix([])

In [3]:
def from_orbit_to_geostat_crd_mat(_orbit_crd):
    crd = Matrix(_orbit_crd)
    rot_z_1 =rot_mat('z', psi)
    rot_x = rot_mat('x', theta)
    rot_z_2 = rot_mat('z', phi)
    
    crd = crd * (rot_z_1 * rot_x * rot_z_2)
    
    return crd

Посмотрим промежуточный вид конечной формулы:

In [4]:
var("a e_orb t")  # создание символьных переменных
crd_orbital = satellite_pos_orbital_crd(_orbit_a=a, _orbit_e=e_orb, _Xi=t)
print("Орбитальные координаты в момент времени t:")
show(crd_orbital[0])
show(crd_orbital[1])
show(crd_orbital[2])

NameError: name 'var' is not defined

In [5]:
var("psi theta phi")
crd_geostat = from_orbit_to_geostat_crd_mat(crd_orbital)[0]
print("Геостац. координаты ИСЗ в момент времени t:")
show(crd_geostat[0])
show(crd_geostat[1])
show(crd_geostat[2])

NameError: name 'var' is not defined

Формула готова, но её еще нужно упростить и сгруппировать

In [6]:
print("Автоматическое упрощение:")
show(crd_geostat[0].simplify_trig())
show(crd_geostat[1].simplify_trig())
show(crd_geostat[2].simplify_trig())

Автоматическое упрощение:


NameError: name 'show' is not defined

Результат не самый оптимальный в текущем случае, потому придется доработать вручную:

In [7]:
def from_orbit_to_geostat_crd(_orbit_crd):
    crd = Matrix(_orbit_crd)
    x = crd[0][0]
    y = crd[0][1]
    z = crd[0][2]
    
    X =\
            - x\
                *(
                        cos(theta)*sin(phi)*sin(psi) 
                    -	cos(phi)*cos(psi)
                )\
            - y\
                *(
                        cos(psi)*cos(theta)*sin(phi) 
                    +	cos(phi)*sin(psi)
                )\
            + z*sin(phi)*sin(theta)


    Y = \
         	x\
            *(
                    cos(phi)*cos(theta)*sin(psi) \
                +	cos(psi)*sin(phi)
            )\
        +	y\
            *(
                    cos(phi)*cos(psi)*cos(theta) \
                -	sin(phi)*sin(psi)
            )\
        -	z*cos(phi)*sin(theta) 

    Z = 	(\
                    x*sin(psi)\
                +	y*cos(psi)
            )*sin(theta)\
        +	z*cos(theta)
        
    return [X, Y, Z]

In [8]:
def satellite_pos_geostat_crd():    
    x = \
        - a	*sqrt(1 - e_orb**2)*sin(Xi)\
            *(
                    cos(psi)*cos(theta)*sin(phi)\
                +	cos(phi)*sin(psi)
             )\
        + a	*(cos(Xi) - e_orb)\
            *(
                -	cos(theta)*sin(phi)*sin(psi)\
                +	cos(phi)*cos(psi)
            )

    y = \
            a	*sqrt(1 - e_orb**2)*sin(Xi)\
                *(
                        cos(phi)*cos(psi)*cos(theta)\
                    - 	sin(phi)*sin(psi)
                )\
            + a	*(cos(Xi) - e_orb)\
                *(
                        cos(phi)*cos(theta)*sin(psi)\
                    +	cos(psi)*sin(phi)
                )

    z = \
            a	*sin(theta)\
                *(\
                        sqrt(1 - e_orb**2)*cos(psi)*sin(Xi)\
                    + 	(cos(Xi) - e_orb)*sin(psi)\
                )
        
    return [x, y, z]

Теперь необходимо удостовериться, что при упрощении вручную ничего не потерялось и формула корректна.

In [9]:
# Текущая формула берет символьные переменные из глобальной области видимости
# потому придется сделать так:
Xi = t
# теперь получим координаты:
crd_geostat2 = satellite_pos_geostat_crd()
for i in range(0, 3):
    show(crd_geostat2[i])

NameError: name 't' is not defined

<h1>Сверим формулы:</h1>

In [10]:
for i in range(0, 3):
    show((crd_geostat2[i] - crd_geostat[i]).simplify_trig())

NameError: name 'show' is not defined

Следовательно в формуле ошибки нет и можно пользоваться satellite_pos_geostat_crd